### Machine Learning


In [3]:
import sys
import numpy as np
import pandas as pd
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pickle

In [4]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Pandas       :', pd.__version__)
print('Sklearn      :', sklearn.__version__)

Python       : 3.8.3 (default, Jul  2 2020, 17:30:36) [MSC v.1916 64 bit (AMD64)]
Numpy        : 1.18.5
Pandas       : 1.0.5
Sklearn      : 0.23.1


In [5]:
np.random.seed(0)

In [4]:
!pip freeze > requirements.txt

In [ ]:
DATA_DIR = '../input/findata-creditscoring/'
train = pd.read_csv(DATA_DIR + 'credit_train.csv')
test = pd.read_csv(DATA_DIR + 'credit_test.csv')
sample_submission = pd.read_csv(DATA_DIR + 'credit__sample.csv')
train.drop_duplicates(keep='first', inplace = True)
train['Loan Status'] = train['Loan Status'].map({"Fully Paid": 1, "Charged Off": 0})
train.drop(['Loan ID','Customer ID',], axis = 1, inplace=True)
test.drop(['Loan ID','Customer ID',], axis = 1, inplace=True)

In [ ]:
y = train['Loan Status'].values     
X = train.drop(['Loan Status'], axis=1)

In [12]:
from AutoML import AutoMLClassifier 

In [8]:
model = AutoMLClassifier()
model.fit(X, y)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [9]:
model.best_pipeline

{'preprocessor__numerical__scaler': None,
 'preprocessor__numerical__cleaner__strategy': 'median',
 'feature_selector__k': 41,
 'estimator__n_estimators': 275,
 'estimator__criterion': 'gini',
 'estimator': RandomForestClassifier(n_estimators=275, random_state=0)}

In [10]:
model.best_score

0.8007796402517124

In [11]:
with open('RF.pkl', 'wb') as output:
       pickle.dump(model, output)

In [12]:
submission = sample_submission
submission['Loan Status'] = model.predict(test)
submission['Loan Status'] = sample_submission['Loan Status'].map({1 : "Fully Paid",  0 : "Charged Off"})
submission.to_csv('RF.csv', index=False)

In [13]:
submission.head()

,Loan ID,Loan Status
0,f738779f-c726-40dc-92cf-689d73af533d,Fully Paid
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,Fully Paid
2,f7744d01-894b-49c3-8777-fc6431a2cff1,Fully Paid
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,Fully Paid
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,Fully Paid


### Выводы

1. Реализована библиотека, автоматизирующая последовательные этапы конвейера машинного обучения:
- заполнение пропусков в данных
- кодирование категориальных переменных
- масштабирование числовых переменных
- подбор признаков.
- выбор модели
- настройку гиперпараметров

2. Методом случайного решетчатого поиска с кросс-валидацией выбраны оптимальные гиперпараметры машинного обучения:
- без масштабирования числовых признаков
- алгоритм заполнения пропусков - медиана
- количество выбранных для обучения модели признаков - 41 шт.
- алгоритм ML - "случайный лес"
- количество деревьев - 275 шт.
- критерий информативности - Джини

3. Обучение указанной модели ML позволило достичь Accuracy, составляющей более 0.8.

4. На тестовых данных, предложенных на соревновании [Kaggle](https://www.kaggle.com/c/findata-creditscoring/leaderboard), значение Accuracy - 0.82266 (по состоянию на 05.06.2021 - 17 место из 269 (TOP7%), nickname - Evgeniy Grinkin)